In [44]:
# число групп на курсе на факультете (10), число преподавателей (15), какие дисциплины изучают студенты групп в текущем семестре и количество часов, отводимое на их изучение, количество недель в семестре (16), аудитории в которых факультет может проводить занятия, вместимость аудиторий.
# Штрафы, начисляемые за недостатки в расписании
# групп:
# За «окно» в расписании: 20
# За каждое «окно» сверх одного: 40
# Пустая пара в начале дня: 5
# За 3 занятия оного типа по одной и той же дисциплине в течение дня: 20
# За каждое занятие одного типа сверх имеющихся 3 по той же дисциплине в течение дня: 100
# Штрафы, начисляемые за недостатки в расписании преподавателей:
# За наличие пар в выходной или праздничный день: 20
# За каждое «окно» сверх одного: 50
# Не предусмотрено время перехода между далеко расположенными аудиториями: 60
# За каждую лишнюю пару сверх максимального числа пар в день: 10
# Общие штрафы:
# Непопадание одного занятия (лекции, практики, семинара) в сетку расписания: 150

In [46]:
NUM_OF_GROUPS = 8
NUM_OF_TEACHERS = 4
DISCIPLINES_TIME = {
    'Математика': 80,
    'Программирование': 125,
    'Физика': 60,
    'Анализ бизнес-процессов': 80,
    'Физкультура': 60,
    'Информатика': 100
}
COUNT_OF_WEEKS = 4
NUM_DAYS_PER_WEEK = 5
NUM_PAIRS_PER_DAY = 6 
MAX_GROUPS_PER_AUDITION = {
    "A101": 2,
    "A102": 2,
    "A103": 4,
    "A104": 4,
    "A108": 8
}
AUDITIONS_TIME_DESTINATIONS = {
    "A101" : {
        "A102": 10,
        "A103": 10,
        "A104": 20,
        "A108": 30,
    },
    "A102" : {
        "A101": 10,
        "A103": 20,
        "A104": 10,
        "A108": 20
    },
    "A103" : {
        "A101": 10,
        "A102": 20,
        "A104": 10,
        "A108": 20
    },
    "A104" : {
        "A101": 20,
        "A102": 10,
        "A104": 10,
        "A108": 10
    },
    "A108" : {
        "A101": 30,
        "A102": 20,
        "A103": 20,
        "A104": 10,
    }
}



НЕДЕЛЯ -> ДЕНЬ -> ПАРА -> АУДИТОРИЯ -> ГРУППА -> ДИСЦИПЛИНА -> ПРЕПОД -> ТИП ПАРЫ [0; 1]
- 0 - ничего нет
- 1 - установлено 

In [47]:
import pandas as pd
import numpy as np

# Создаем n-мерный массив
arr = np.random.rand(3, 4, 5)

# Преобразуем массив в двумерный массив
reshaped_arr = arr.reshape(-1, 3)

# Создаем DataFrame из двумерного массива
df = pd.DataFrame(reshaped_arr)

# Выводим DataFrame
print(df)

           0         1         2
0   0.093148  0.355193  0.075590
1   0.463243  0.722193  0.784873
2   0.101906  0.807948  0.931409
3   0.849016  0.894945  0.245812
4   0.559209  0.988078  0.327588
5   0.056034  0.660391  0.221451
6   0.498320  0.044823  0.938693
7   0.430417  0.753027  0.265807
8   0.774983  0.845965  0.403199
9   0.914345  0.500948  0.520882
10  0.486406  0.785142  0.990867
11  0.311557  0.953407  0.376587
12  0.522336  0.152175  0.628172
13  0.465754  0.147253  0.226977
14  0.684325  0.035732  0.801608
15  0.828015  0.304819  0.823974
16  0.792421  0.371979  0.997023
17  0.330431  0.536615  0.728326
18  0.273565  0.673915  0.627040
19  0.350230  0.929876  0.614200


In [48]:
def pso(
        func_to_optimize,
        *,
        lb, 
        ub,
        kp = 0.5,
        kg = 0.5,
        maxiter = 500, 
        swarm_size = 100,  
        min_f_delta = 1e-5,
        min_step_delta = 1e-5,
        family_size = 1
    ):
    """
    """
    
    # Задание параметров алгоритма роя частиц
    optim = func_to_optimize    # Функция, которую собираемся оптимизировать
    S = swarm_size              # Размер роя частиц (их кол-во)
    D = len(lb)                 # Размерность частицы (кол-во параметров частицы)
    x = np.random.rand(S, D)    # рандомные точки пространства
    v = np.zeros_like(x)        # скорость частиц
    p = np.zeros_like(x)        # лучшие точки итерации
    vmax = np.abs(ub - lb)      # максимальная скорость сдвига
    vmin = -vmax                # минимальная скорость сдвига
    g = []                      # Глобальная лучшая точка
    
    
    fx = np.zeros(S)            # Вычисляемое значение на основе частицы
    fp = np.ones(S) * np.inf    # Значения функций в лучших точках
    fg = np.inf                 # Значение лучшей точки
    
    
    # Инициализация начальных значений частиц
    x = lb + x * (ub - lb)
    
    # Инициализация начальных скоростей
    v = vmin + np.random.rand(S, D) * (vmax - vmin)
    
    # Выполняем первичное приближение
    for i in range(S):
        fx[i] = optim(x[i, :])
        
    i_update = fx < fp 
    p[i_update, :] = x[i_update, :].copy()
    fp[i_update] = fx[i_update].copy()
    
    i_best = np.argmin(fp)
    if fp[i_best] < fg:
        fg = fp[i_best]
        g = p[i_best, :].copy()
    else:
        g = x[0, :].copy()
        
    it = 1
    
    # if family_size > 1:
    #     v_f = [v.copy() for i in range(family_size)]
    #     x_f = [x.copy() for i in range(family_size)]
    #     g_f = [g.copy() for i in range(family_size)]
    
    
    while it <= maxiter:
        
        rp = np.random.uniform(size=(S, D))
        rg = np.random.uniform(size=(S, D))
        
        # Корректировка показателей скоростей
        v = v + kp * rp * (p - x) + kg * rg * (g - x)
        
        # Определение новых позиций частиц
        x = x + v
        
        # Корректировка значений, которые могли вылететь за заданные границы
        lower_mask = x < lb
        upper_mask = x > ub
        
        x = x * (~np.logical_or(lower_mask, upper_mask)) + lb * lower_mask + ub * upper_mask
        
        for i in range(S):
            fx[i] = optim(x[i, :])
        
        i_update = fx < fp 
        p[i_update, :] = x[i_update, :].copy()
        fp[i_update] = fx[i_update].copy()
        
        i_best = np.argmin(fp)
        print(f"Итерация #{it}: Лучшая частица {p[i_best, :]} ({fp[i_best]:5f})")
        if fp[i_best] < fg:
            
            if np.abs(fg - fp[i_best]) <= min_f_delta:
                print("КОНЕЦ (ПРИЧИНА: изменение лучшего значения меньше {})".format(min_f_delta))
                return p[i_best, :].copy(), fp[i_best]
            
            elif np.sqrt(np.sum((g - p[i_best, :])**2)) <= min_step_delta:
                print("КОНЕЦ (ПРИЧИНА: разница между соседними лучшими значениями меньше {})".format(min_step_delta))
                return p[i_best, :].copy(), fp[i_best]
                
            else:            
                fg = fp[i_best]
                g = p[i_best, :].copy()
        
        it+=1
        
    print('Цикл окончен')
    return g, fg

In [49]:
def func(arg_x):
    x = arg_x[0]
    y = arg_x[1]
    
    return 0.26 * (x**2 + y**2) - 0.48 * x * y

In [50]:
xy, ans = pso(
    func, 
    lb=np.array([-4.5, -4.5]),
    ub=np.array([4.5, 4.5]),
    swarm_size=300,
    maxiter=500)

print(np.round(xy), round(ans))

Итерация #1: Лучшая частица [-0.24669994 -0.26529111] (0.002708)
Итерация #2: Лучшая частица [-0.24669994 -0.26529111] (0.002708)
Итерация #3: Лучшая частица [-0.24669994 -0.26529111] (0.002708)
Итерация #4: Лучшая частица [-0.24669994 -0.26529111] (0.002708)
Итерация #5: Лучшая частица [-0.24669994 -0.26529111] (0.002708)
Итерация #6: Лучшая частица [0.17345444 0.11168316] (0.001767)
Итерация #7: Лучшая частица [0.17345444 0.11168316] (0.001767)
Итерация #8: Лучшая частица [0.17345444 0.11168316] (0.001767)
Итерация #9: Лучшая частица [0.10922984 0.07228367] (0.000671)
Итерация #10: Лучшая частица [0.10922984 0.07228367] (0.000671)
Итерация #11: Лучшая частица [0.10922984 0.07228367] (0.000671)
Итерация #12: Лучшая частица [0.10922984 0.07228367] (0.000671)
Итерация #13: Лучшая частица [0.10922984 0.07228367] (0.000671)
Итерация #14: Лучшая частица [0.10922984 0.07228367] (0.000671)
Итерация #15: Лучшая частица [0.05070377 0.05621367] (0.000122)
Итерация #16: Лучшая частица [0.0507037

# Реализация роя частиц

In [51]:
class ParticleSwarm:
    def __init__(self, optim, ub, lb, kp, kg, swarm_size):
        self.optim = optim
        self.lb = lb
        self.ub = ub
        self.kp = kp
        self.kg = kg
        
        self.S = swarm_size
        self.D = len(lb)
        self.x = np.random.rand(self.S, self.D)    # рандомные точки пространства
        self.v = np.zeros_like(self.x)             # скорость частиц
        self.p = np.zeros_like(self.x)             # лучшие точки итерации
        self.vmax = np.abs(ub - lb)                # максимальная скорость сдвига
        self.vmin = -self.vmax                     # минимальная скорость сдвига
        self.g = []                                # Глобальная лучшая точка


        self.fx = np.zeros(self.S)                 # Вычисляемое значение на основе частицы
        self.fp = np.ones(self.S) * np.inf         # Значения функций в лучших точках
        self.fg = np.inf                           # Значение лучшей точки
        
        # Инициализация начальных значений частиц
        self.x = self.lb + self.x * (self.ub - self.lb)

        # Инициализация начальных скоростей
        self.v = self.vmin + np.random.rand(self.S, self.D) * (self.vmax - self.vmin)

        # Выполняем первичное приближение
        for i in range(self.S):
            self.fx[i] = self.optim(self.x[i, :])

        i_update = self.fx < self.fp 
        self.p[i_update, :] = self.x[i_update, :].copy()
        self.fp[i_update] = self.fx[i_update].copy()

        i_best = np.argmin(self.fp)
        if self.fp[i_best] < self.fg:
            self.fg = self.fp[i_best]
            self.g = self.p[i_best, :].copy()
        else:
            self.g = self.x[0, :].copy()

    
    def step(self, it):
        rp = np.random.uniform(size=(self.S, self.D))
        rg = np.random.uniform(size=(self.S, self.D))
        
        # Корректировка показателей скоростей
        self.v = self.v + self.kp * rp * (self.p - self.x) + self.kg * rg * (self.g - self.x)
        
        # Определение новых позиций частиц
        self.x = self.x + self.v
        
        # Корректировка значений, которые могли вылететь за заданные границы
        lower_mask = self.x < self.lb
        upper_mask = self.x > self.ub
        
        self.x = self.x * (~np.logical_or(lower_mask, upper_mask)) + self.lb * lower_mask + self.ub * upper_mask
        
        for i in range(self.S):
            self.fx[i] = self.optim(self.x[i, :])
        
        i_update = self.fx < self.fp 
        self.p[i_update, :] = self.x[i_update, :].copy()
        self.fp[i_update] = self.fx[i_update].copy()
        
        self.i_best = np.argmin(self.fp)
        print(f"Итерация #{it}: Лучшая частица {self.p[self.i_best, :]} ({self.fp[self.i_best]:5f})")
            
            
    def validate_ans_setup_g(self, min_f_delta, min_step_delta):
        stop_flg = False
        if self.fp[self.i_best] < self.fg:
            if np.abs(self.fg - self.fp[self.i_best]) <= min_f_delta:
                print("КОНЕЦ (ПРИЧИНА: изменение лучшего значения меньше {})".format(min_f_delta))
                stop_flg = True
            
            elif np.sqrt(np.sum((self.g - self.p[self.i_best, :])**2)) <= min_step_delta:
                print("КОНЕЦ (ПРИЧИНА: разница между соседними лучшими значениями меньше {})".format(min_step_delta))
                stop_flg = True                
                    
            self.fg = self.fp[self.i_best]
            self.g = self.p[self.i_best, :].copy()
                
                
        return stop_flg
    
    
    def answer(self):
        return self.g, self.fg
        

# Реализация алгоритма роя частиц с возможным выбором островов

In [52]:
class PSOAlgo:
    def __init__(self, func_to_optimize, *, lb, ub, kp=0.5, kg=0.5, swarm_size = 100):
        self.optim = func_to_optimize
        self.lb = lb
        self.ub = ub
        self.kp = kp
        self.kg = kg
        self.swarm_size = swarm_size
        
        
    def _build_swarm(self, swarm_size):
        return ParticleSwarm(
            optim = self.optim,
            lb = self.lb,
            ub = self.ub,
            kp=self.kp,
            kg=self.kg,
            swarm_size = swarm_size
        )
    
        
    def init_swarm(self, family_size):
        if family_size > 1:
            self.sw_family = [self._build_swarm(self.swarm_size) for i in range(family_size)]
        else:
            self.sw = self._build_swarm(self.swarm_size)
            
    
    def run(self, maxiter=100, min_f_delta=1e-5, min_step_delta=1e-5):
        it = 1
        while it < maxiter:
            if hasattr(self, "sw_family"):
                for swarm in self.sw_family:
                    swarm.step(it)
            else:
                self.sw.step(it)
                
            if hasattr(self, "sw_family"):
                stop_flg = any([swarm.validate_ans_setup_g(min_f_delta, min_step_delta) for swarm in self.sw_family])
            else:
                stop_flg = self.sw.validate_ans_setup_g(min_f_delta, min_step_delta)
            
            if stop_flg:
                break
            
            #Передача, блин, лучшего результата всем роям, блин
            if it % 10 == 0 and hasattr(self, "sw_family"):
                fgs = [swarm.fg for swarm in self.sw_family]
                i_best_fg = np.argmin(fgs)
                
                for swarm in self.sw_family:
                    swarm.fg = self.sw_family[i_best_fg].fg
                    swarm.g = self.sw_family[i_best_fg].g.copy()
            
            it += 1
        else:
            print('Цикл окончен')
        

    def get_answer(self):
        if hasattr(self, "sw_family"):
            fgs = [swarm.fg for swarm in self.sw_family]
            i_best_fg = np.argmin(fgs)
            return self.sw_family[i_best_fg].answer()
        else:
            return self.sw.answer()

In [53]:
pso_alg = PSOAlgo(
    func,
    lb=np.array([-4.5, -4.5]),
    ub=np.array([4.5, 4.5]),
    swarm_size=300)

In [54]:
pso_alg.init_swarm(2)

In [55]:
pso_alg.run()

Итерация #1: Лучшая частица [0.58862033 0.56628857] (0.013463)
Итерация #1: Лучшая частица [-0.08637178  0.11281669] (0.009926)
Итерация #2: Лучшая частица [0.58862033 0.56628857] (0.013463)
Итерация #2: Лучшая частица [-0.08637178  0.11281669] (0.009926)
Итерация #3: Лучшая частица [0.58862033 0.56628857] (0.013463)
Итерация #3: Лучшая частица [-0.08637178  0.11281669] (0.009926)
Итерация #4: Лучшая частица [0.3727376  0.39506441] (0.006020)
Итерация #4: Лучшая частица [-0.08637178  0.11281669] (0.009926)
Итерация #5: Лучшая частица [0.3727376  0.39506441] (0.006020)
Итерация #5: Лучшая частица [-0.08637178  0.11281669] (0.009926)
Итерация #6: Лучшая частица [0.05918878 0.12833191] (0.001547)
Итерация #6: Лучшая частица [-0.20240606 -0.20360956] (0.001649)
Итерация #7: Лучшая частица [0.05918878 0.12833191] (0.001547)
Итерация #7: Лучшая частица [-0.20240606 -0.20360956] (0.001649)
Итерация #8: Лучшая частица [0.05918878 0.12833191] (0.001547)
Итерация #8: Лучшая частица [-0.20240606 

In [56]:
xy, ans = pso_alg.get_answer()

print(np.round(xy), round(ans))

[-0. -0.] 0


неделя-день недели-пара-аудитория-предмет-тип занятия

In [57]:
num_weeks = 1
week_days = ['Понедельник', 'Вторник',
            #  'Среда', 'Четверг', 'Пятница'
             ]
num_pairs = 2
audit = ['a101', 'a102', 'a103']
disciplines = ['Матеша', 'Физика', 'Русский', 'Окно']
types_of_lesson = ['Лекция', 'Практика']

In [58]:
from itertools import product

In [59]:
for e in product(range(num_weeks), week_days, range(num_pairs), audit, disciplines, types_of_lesson):
    print(f"Неделя #{e[0] + 1} - {e[1]} - Пара #{e[2] + 1} - {e[3]} - {e[4]} - {e[5]}")

Неделя #1 - Понедельник - Пара #1 - a101 - Матеша - Лекция
Неделя #1 - Понедельник - Пара #1 - a101 - Матеша - Практика
Неделя #1 - Понедельник - Пара #1 - a101 - Физика - Лекция
Неделя #1 - Понедельник - Пара #1 - a101 - Физика - Практика
Неделя #1 - Понедельник - Пара #1 - a101 - Русский - Лекция
Неделя #1 - Понедельник - Пара #1 - a101 - Русский - Практика
Неделя #1 - Понедельник - Пара #1 - a101 - Окно - Лекция
Неделя #1 - Понедельник - Пара #1 - a101 - Окно - Практика
Неделя #1 - Понедельник - Пара #1 - a102 - Матеша - Лекция
Неделя #1 - Понедельник - Пара #1 - a102 - Матеша - Практика
Неделя #1 - Понедельник - Пара #1 - a102 - Физика - Лекция
Неделя #1 - Понедельник - Пара #1 - a102 - Физика - Практика
Неделя #1 - Понедельник - Пара #1 - a102 - Русский - Лекция
Неделя #1 - Понедельник - Пара #1 - a102 - Русский - Практика
Неделя #1 - Понедельник - Пара #1 - a102 - Окно - Лекция
Неделя #1 - Понедельник - Пара #1 - a102 - Окно - Практика
Неделя #1 - Понедельник - Пара #1 - a103 - М

In [60]:
set_of_all_lessons = list(product(range(num_weeks), week_days, range(num_pairs), audit, disciplines, types_of_lesson))

In [61]:
z = np.zeros((len(set_of_all_lessons), len(set_of_all_lessons)))

In [62]:
data = pd.DataFrame(z.copy(), columns=set_of_all_lessons, index=set_of_all_lessons)
data

,"(0, Понедельник, 0, a101, Матеша, Лекция)","(0, Понедельник, 0, a101, Матеша, Практика)","(0, Понедельник, 0, a101, Физика, Лекция)","(0, Понедельник, 0, a101, Физика, Практика)","(0, Понедельник, 0, a101, Русский, Лекция)","(0, Понедельник, 0, a101, Русский, Практика)","(0, Понедельник, 0, a101, Окно, Лекция)","(0, Понедельник, 0, a101, Окно, Практика)","(0, Понедельник, 0, a102, Матеша, Лекция)","(0, Понедельник, 0, a102, Матеша, Практика)",...,"(0, Вторник, 1, a102, Окно, Лекция)","(0, Вторник, 1, a102, Окно, Практика)","(0, Вторник, 1, a103, Матеша, Лекция)","(0, Вторник, 1, a103, Матеша, Практика)","(0, Вторник, 1, a103, Физика, Лекция)","(0, Вторник, 1, a103, Физика, Практика)","(0, Вторник, 1, a103, Русский, Лекция)","(0, Вторник, 1, a103, Русский, Практика)","(0, Вторник, 1, a103, Окно, Лекция)","(0, Вторник, 1, a103, Окно, Практика)"
"(0, Понедельник, 0, a101, Матеша, Лекция)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(0, Понедельник, 0, a101, Матеша, Практика)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(0, Понедельник, 0, a101, Физика, Лекция)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(0, Понедельник, 0, a101, Физика, Практика)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(0, Понедельник, 0, a101, Русский, Лекция)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(0, Вторник, 1, a103, Физика, Практика)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(0, Вторник, 1, a103, Русский, Лекция)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(0, Вторник, 1, a103, Русский, Практика)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(0, Вторник, 1, a103, Окно, Лекция)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
# бан за невозможные одновременные пары
for para in data.columns:
    
    same_banned_pairs = product((para[0],), (para[1],), (para[2],), (para[3],), set(disciplines).difference({para[4]}), types_of_lesson)
    another_type_pair = para[:-1] + tuple(set(types_of_lesson).difference({para[-1]}))
    
    data[para][another_type_pair] += 100
    
    for p in same_banned_pairs:
        data[para][p] += 500

/var/folders/h7/dv3s_s754013wnc2h_5m8nd00000gn/T/ipykernel_28836/3567322100.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data[para][another_type_pair] += 100
/var/folders/h7/dv3s_s754013wnc2h_5m8nd00000gn/T/ipykernel_28836/3567322100.

In [65]:
# бан за окна 
banned_window_pairs = product(range(num_weeks), week_days, range(num_pairs), audit, ('Окно',), types_of_lesson)

for para in banned_window_pairs:
    data[para] += 100

In [66]:
# бан за слишком долгие переходы
dest = {
    "a101": {
        "a102": 10,
        "a103": 30
    },
    "a102": {
        "a101": 10,
        "a103": 10,
    },
    "a103": {
        "a101": 30,
        "a102": 10
    }
}

for para in data.columns:
    if para[2] == num_pairs - 1:
        continue
    
    next_pairs = product(
        (para[0],), 
        (para[1],),
        (para[2] + 1,),
        set(audit).difference({para[3]}),
        set(disciplines).difference({'Окно'}),
        types_of_lesson
        )
    
    for next_para in next_pairs:
        if dest[para[3]][next_para[3]] > 10:
            data[para][next_para] += 150

/var/folders/h7/dv3s_s754013wnc2h_5m8nd00000gn/T/ipykernel_28836/1131043028.py:32: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data[para][next_para] += 150
/var/folders/h7/dv3s_s754013wnc2h_5m8nd00000gn/T/ipykernel_28836/1131043028.py:32: 

In [67]:
#Пустая пара в начале дня
first_pairs_windows = product(
    range(num_weeks),
    week_days,
    (0, ),
    audit,
    ('Окно', ),
    types_of_lesson
)

for para in first_pairs_windows:
    data[para] += 200

In [68]:
data

,"(0, Понедельник, 0, a101, Матеша, Лекция)","(0, Понедельник, 0, a101, Матеша, Практика)","(0, Понедельник, 0, a101, Физика, Лекция)","(0, Понедельник, 0, a101, Физика, Практика)","(0, Понедельник, 0, a101, Русский, Лекция)","(0, Понедельник, 0, a101, Русский, Практика)","(0, Понедельник, 0, a101, Окно, Лекция)","(0, Понедельник, 0, a101, Окно, Практика)","(0, Понедельник, 0, a102, Матеша, Лекция)","(0, Понедельник, 0, a102, Матеша, Практика)",...,"(0, Вторник, 1, a102, Окно, Лекция)","(0, Вторник, 1, a102, Окно, Практика)","(0, Вторник, 1, a103, Матеша, Лекция)","(0, Вторник, 1, a103, Матеша, Практика)","(0, Вторник, 1, a103, Физика, Лекция)","(0, Вторник, 1, a103, Физика, Практика)","(0, Вторник, 1, a103, Русский, Лекция)","(0, Вторник, 1, a103, Русский, Практика)","(0, Вторник, 1, a103, Окно, Лекция)","(0, Вторник, 1, a103, Окно, Практика)"
"(0, Понедельник, 0, a101, Матеша, Лекция)",0.0,100.0,500.0,500.0,500.0,500.0,900.0,900.0,0.0,0.0,...,200.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0
"(0, Понедельник, 0, a101, Матеша, Практика)",100.0,0.0,500.0,500.0,500.0,500.0,900.0,900.0,0.0,0.0,...,200.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0
"(0, Понедельник, 0, a101, Физика, Лекция)",500.0,500.0,0.0,100.0,500.0,500.0,900.0,900.0,0.0,0.0,...,200.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0
"(0, Понедельник, 0, a101, Физика, Практика)",500.0,500.0,100.0,0.0,500.0,500.0,900.0,900.0,0.0,0.0,...,200.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0
"(0, Понедельник, 0, a101, Русский, Лекция)",500.0,500.0,500.0,500.0,0.0,100.0,900.0,900.0,0.0,0.0,...,200.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(0, Вторник, 1, a103, Физика, Практика)",0.0,0.0,0.0,0.0,0.0,0.0,400.0,400.0,0.0,0.0,...,200.0,200.0,500.0,500.0,100.0,0.0,500.0,500.0,700.0,700.0
"(0, Вторник, 1, a103, Русский, Лекция)",0.0,0.0,0.0,0.0,0.0,0.0,400.0,400.0,0.0,0.0,...,200.0,200.0,500.0,500.0,500.0,500.0,0.0,100.0,700.0,700.0
"(0, Вторник, 1, a103, Русский, Практика)",0.0,0.0,0.0,0.0,0.0,0.0,400.0,400.0,0.0,0.0,...,200.0,200.0,500.0,500.0,500.0,500.0,100.0,0.0,700.0,700.0
"(0, Вторник, 1, a103, Окно, Лекция)",0.0,0.0,0.0,0.0,0.0,0.0,400.0,400.0,0.0,0.0,...,200.0,200.0,500.0,500.0,500.0,500.0,500.0,500.0,200.0,300.0


In [ ]:
def penalty_func_group(x):
    x = np.round(x)
    result_penalty = 0

    # штрафы в рамках всего расписания (оценка относительно <> 0 выбора)
    group_data = data.loc[:, x > 0]
    for lesson in group_data.columns:
        result_penalty += group_data[lesson].sum()

    # штрафы в рамках группового расписания (по конкретной группе)
    for group in range(len(groups)):
        def_group_data = data.loc[:, x == group + 1]
        gr_lessons = def_group_data.columns

        if len(gr_lessons) == 0:
            result_penalty += 10000
        else:
            for group_1, group_2 in combinations(gr_lessons, 2):
                result_penalty += def_group_data[group_1][group_2]

    return result_penalty
